# TUGAS BESAR ANALISIS JEJARING SOSIAL
KELOMPOK H:

DHAFA NUR FADHILAH (1301213263)

PUTRI MAHARANI (1301213093)

RATIN KANI (1301213269)

#Deskripsi Tugas Besar

Tugas besar ini bertujuan untuk mengidentifikasi komunitas politik pada platform sosial media X. Data yang digunakan adalah dataset berasal dari crawling data menggunakan tweet-harvest. Dataset dapat diakses melalui tautan berikut [tautan_dataset](https://drive.google.com/file/d/1Gs-QVz1XYQ1l1xQW2g73N3OjoR6QXkcM/view?usp=sharing).

Dataset ini berisi jumlah like, reply, retweet, dan mention yang diberikan oleh pengguna dengan username tertentu terhadap suatu post. Data tersebut kemudian diolah untuk selanjutnya dijadikan edges supaya bisa di deteksi komunitasnya oleh metode Louvain, Greedy, dan  Label Propogation.

#Load Data

In [ ]:
import json
import pandas as pd
import networkx as nx
from community import community_louvain
from networkx.algorithms.community import greedy_modularity_communities
from networkx.algorithms.community import label_propagation_communities
from networkx.algorithms.community.quality import modularity

In [ ]:
# Clone the dataset from github and read the csv
!git clone https://github.com/Dhafanur25/data_ajs.git

df = pd.read_csv('/content/data_ajs/gibranPrabowo.csv')
df.tail()

fatal: destination path 'data_ajs' already exists and is not an empty directory.


,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
2426,1860241809794040097,Mon Nov 25 00:03:17 +0000 2024,0,@OjolNyambi Wkwkwkwk.dasar pasukan sakit hati....,1860836617839354237,NaN,OjolNyambi,in,NaN,0,0,0,https://x.com/SiauwYakuza/status/1860836617839...,1066882757907767296,SiauwYakuza
2427,1860584347440689159,Mon Nov 25 00:03:11 +0000 2024,0,@N74ms @jansen_jsp @SBYudhoyono @prabowo giman...,1860836592744911039,NaN,N74ms,in,NaN,0,0,0,https://x.com/484nX/status/1860836592744911039,1714692943628668928,484nX
2428,1860833622649082316,Mon Nov 25 00:02:42 +0000 2024,1,@QueenSilence_ Dengerlah jeritan rakyatmu ini ...,1860836474608120213,NaN,QueenSilence_,in,NaN,0,0,0,https://x.com/dondongmerah729/status/186083647...,1649724978110271490,dondongmerah729
2429,1860834378076881136,Mon Nov 25 00:02:31 +0000 2024,2,@kosongwolu_08 @prabowo Orang munafik biasanya...,1860836425937362964,NaN,kosongwolu_08,in,Indonesia,0,1,0,https://x.com/AlmaghribiS/status/1860836425937...,2717547397,AlmaghribiS
2430,1860708594238570820,Mon Nov 25 00:02:07 +0000 2024,1,@vanc1Bozz @ListyoSigitP @prabowo Sudah saatny...,1860836324280016980,NaN,vanc1Bozz,in,Indonesia,0,0,0,https://x.com/dwiwarna9/status/186083632428001...,963089861111513088,dwiwarna9


In [ ]:
# Extract Edges
edges = []
for index, row in df.iterrows():
    user = row['username']
    mentions = [mention.strip('@') for mention in row['full_text'].split() if mention.startswith('@')]
    for mention in mentions:
        edges.append((user, mention))

# Create Graph
G = nx.Graph()
G.add_edges_from(edges)

In [ ]:
# Export Nodes dan print hasil
def export_nodes(graph, partition, algorithm_name):
    nodes_data = []
    for node in graph.nodes():
        nodes_data.append({
            'id': node,
            'label': node,
            'community': partition.get(node, -1)
        })
    return pd.DataFrame(nodes_data)

#Metode Louvain

In [ ]:
# Louvain Method
louvain_partition = community_louvain.best_partition(G)

louvain_modularity = modularity(G, [{node for node in louvain_partition if louvain_partition[node] == c} for c in set(louvain_partition.values())])
print(f"Louvain Modularity: {louvain_modularity}")

louvain_nodes = export_nodes(G, louvain_partition, 'Louvain')
louvain_nodes

Louvain Modularity: 0.6626761836031295


,id,label,community
0,Istvannandi,Istvannandi,0
1,yaniarsim,yaniarsim,0
2,gibran_tweet,gibran_tweet,0
3,prabowo,prabowo,1
4,ImanoMauren,ImanoMauren,2
...,...,...,...
2394,N74ms,N74ms,4
2395,dondongmerah729,dondongmerah729,1
2396,QueenSilence_,QueenSilence_,1
2397,kosongwolu_08,kosongwolu_08,3


#Metode Greedy

In [ ]:
# Greedy
greedy_communities = list(greedy_modularity_communities(G))
greedy_partition = {node: i for i, community in enumerate(greedy_communities) for node in community}

greedy = modularity(G, [{node for node in greedy_partition if greedy_partition[node] == c} for c in set(greedy_partition.values())])
print(f"Greedy Modularity: {greedy}")

greedy_nodes = export_nodes(G, greedy_partition, 'Greedy')
greedy_nodes

Greedy Modularity: 0.6376876261909598


,id,label,community
0,Istvannandi,Istvannandi,0
1,yaniarsim,yaniarsim,0
2,gibran_tweet,gibran_tweet,0
3,prabowo,prabowo,1
4,ImanoMauren,ImanoMauren,1
...,...,...,...
2394,N74ms,N74ms,2
2395,dondongmerah729,dondongmerah729,1
2396,QueenSilence_,QueenSilence_,1
2397,kosongwolu_08,kosongwolu_08,2


#Metode Label Propogation

In [ ]:
# Label Propagation
lp_communities = list(label_propagation_communities(G))
lp_partition = {node: i for i, community in enumerate(lp_communities) for node in community}

lp_modularity = modularity(G, [{node for node in lp_partition if lp_partition[node] == c} for c in set(lp_partition.values())])
print(f"Label Propagation Modularity: {lp_modularity}")

lp_nodes = export_nodes(G, lp_partition, 'Label Propagation')
lp_nodes

Label Propagation Modularity: 0.5742102285268714


,id,label,community
0,Istvannandi,Istvannandi,0
1,yaniarsim,yaniarsim,0
2,gibran_tweet,gibran_tweet,0
3,prabowo,prabowo,1
4,ImanoMauren,ImanoMauren,2
...,...,...,...
2394,N74ms,N74ms,1
2395,dondongmerah729,dondongmerah729,313
2396,QueenSilence_,QueenSilence_,313
2397,kosongwolu_08,kosongwolu_08,294


#Export hasil nodes dan edges untuk visualisasi

In [ ]:
#Export Edges
edges_data = [{'source': u, 'target': v, 'weight': 1} for u, v in G.edges()]
edges_df = pd.DataFrame(edges_data)

louvain_nodes.to_csv('louvain_nodes.csv', index=False)
greedy_nodes.to_csv('greedy_nodes.csv', index=False)
lp_nodes.to_csv('lp_nodes.csv', index=False)
edges_df.to_csv('edges.csv', index=False)

print("Program selesai")

Program selesai
